## 6.5 Pomiar jakości klastrowania
W przypadku metod klastrowania, możemy rozpatrywać kwestię jakości na dwa sposoby. Jeśli znamy oczekiwany docelowy podział, to możemy sprawdzić czy wygenerowane klastry odpowiadają naszym oczekiwaniom. Jeśli jednak szukamy naturalnego podziału danych, ale nie zakładamy nic na temat podobieństwa poszczególnych obserwacji, to możemy rozpatrywać jakość w kategorii podobieństw wewnątrz klastrów.

### Jakość w przypadku nadzorowanym
Powróćmy do przypadku zbioru danych Titanic. Nauczmy jeszcze raz ten sam model co w poprzednim rozdziale i zbadajmy jaka jest jakość takiego rozwiązania.

In [2]:
from random import random

import pandas as pd

In [3]:
reduced_titanic_df = pd.read_parquet("../data/titanic-reduced.parquet")
reduced_titanic_df.sample(5)

,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Master.,Dr.,Rev.,Names_count
812,0,3,23.0,0,0,7.9250,False,False,False,4
461,0,3,34.0,0,0,8.0500,False,False,False,3
200,0,3,17.0,8,2,69.5500,False,False,False,3
196,0,3,42.0,0,1,8.4042,False,False,False,5
205,0,3,32.0,1,0,15.8500,False,False,False,4


In [4]:
titanic_df = pd.read_parquet("../data/titanic-final.parquet")

In [5]:
titanic_df

,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Mr.,Miss.,Mrs.,Master.,Dr.,Rev.,Names_count,Sex_female
0,0,3,22.0,1,0,7.2500,True,False,False,False,False,False,4,False
1,1,1,38.0,1,0,71.2833,False,False,True,False,False,False,7,True
2,1,3,26.0,0,0,7.9250,False,True,False,False,False,False,3,True
3,1,1,35.0,1,0,53.1000,False,False,True,False,False,False,7,True
4,0,3,35.0,0,0,8.0500,True,False,False,False,False,False,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,0,2,27.0,0,0,13.0000,False,False,False,False,False,True,3,False
883,1,1,19.0,0,0,30.0000,False,True,False,False,False,False,4,True
884,0,3,7.0,1,2,23.4500,False,True,False,False,False,False,4,True
885,1,1,26.0,0,0,30.0000,True,False,False,False,False,False,4,False


### Ponowne nauczenie modelu
Stworzymy ten sam model co poprzednio, zanim jeszcze zaczniemy mierzyć jakość powstałych klientów.

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

In [7]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("clustering", KMeans(n_clusters=2, random_state=2019))
], verbose=True)


In [8]:
import numpy as np

In [9]:
clusters = pipeline.fit_predict(reduced_titanic_df.drop(columns="Survived"))

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing clustering, total=   0.1s


### Jednorodność (ang. Homogeneity)
Jednorodność jest miarą, która pozwala oszacować, czy powstałe klastry są zgodne z oczekiwanym podziałem, tj. czy każdy klaster zawiera tylko elementy pojedynczej klasy. Miara ta jest wartością z przedziału [0, 1], gdzie 1 oznacza perfekcyjne dopasowanie.

In [10]:
from sklearn.metrics import homogeneity_score

In [11]:
homogeneity_score(reduced_titanic_df["Survived"], clusters)

np.float64(0.06785211461108759)

### Kompletność (ang. Completeness)
Kompletność mierzy jakość klastrowania poprzez sprawdzenie czy wszystkie elementy należące do każdej klasy należą równocześnie do tego samego klastra.

In [12]:
from sklearn.metrics import completeness_score

In [13]:
completeness_score(reduced_titanic_df["Survived"], clusters)

np.float64(0.07418491399845831)

### V-measure
Gdybyśmy wykorzystali średnią harmoniczną jednorodności oraz kompletności, to otrzymamy metrykę V-measure.

In [14]:
from sklearn.metrics import v_measure_score

In [15]:
v_measure_score(reduced_titanic_df["Survived"], clusters)

np.float64(0.07087733862518686)

### Pomiar jakości, a uczenie nienadzorowane
Rozpatrując klastrowanie w uczeniu nadzorowanym, jesteśmy w stanie równie dobrze skorzystać z metryk charakterystycznych dla klasyfikacji, żeby zmierzyć skuteczność naszego rozwiązania. W sytuacji iczenia bez podanych etykiet, musimy do problemu podejść w inny sposób i oprzeć pomiar jakości na odległości poszczególnych obserwacji od siebie.

Metody te potrzebują do obliczeń zbioru danych, ale w takiej postaci, w jakiej jest on przekazywany do metody klastrowania.

In [17]:
X = pipeline.named_steps["scaler"].transform(reduced_titanic_df.drop(columns="Survived"))

### Silhouette
Ta metoda pozwala na obliczenie tego, jak bardzo dany obiekt jest podobny do swojego klastra w porównaniu do innych klastrów. Metryka zwraca wartość w przedziale [-1, 1].

In [21]:
from sklearn.metrics import silhouette_score, silhouette_samples

In [22]:
silhouette_samples(X, clusters)

array([ 5.09555525e-01,  1.81085671e-01,  5.48570171e-01,  1.53256600e-01,
        4.82657695e-01,  5.45655446e-01,  2.12260904e-01,  2.04596983e-01,
        1.66660119e-01,  2.67592175e-01,  4.21076548e-01,  1.20612435e-01,
        5.40310105e-01,  1.07794587e-01,  4.35534488e-01, -2.26504535e-02,
        1.92715990e-01,  3.68321645e-01,  1.87532565e-01,  5.52753278e-01,
        2.63610942e-01,  3.31497017e-01,  5.28897448e-01,  3.17304173e-02,
        3.29455629e-01,  4.85390876e-02,  5.36024358e-01,  9.41658107e-02,
        5.52068316e-01,  5.52723407e-01,  1.17053029e-01,  2.41977086e-01,
        5.34885009e-01, -1.32537538e-02,  1.13406210e-01,  1.62215436e-01,
        5.42784845e-01,  5.41757751e-01,  3.65108686e-01,  5.00106646e-01,
        2.42201628e-01, -2.87202787e-02,  1.49476837e-01,  5.37996189e-01,
        5.18623112e-01,  5.11070045e-01,  5.45937209e-01,  4.26413650e-01,
        4.22231971e-01,  1.80261527e-01,  5.41861129e-01,  2.33757337e-01,
       -1.22532477e-01,  

In [23]:
silhouette_score(X, clusters)

np.float64(0.3084774888084638)

### Davies-Boulding score
Metoda Davies-Boulding score wylicza średnie podobieństwo pomiędzy danym klastrem a najbardziej podobnym do niego innym klastrem. Podobieństwem, w tym wypadku, jest stosunek odległości wewnątrz klastra do odległości pomiędzy tymi klastrami.

In [24]:
from sklearn.metrics import davies_bouldin_score

In [26]:
davies_bouldin_score(X, clusters)

np.float64(1.764676965750701)